In [127]:
import numpy as np
import os
import sys
from time import time
from astropy.io import fits

from matplotlib import pyplot as plt



from astropy.cosmology import WMAP9 as cosmo

from scipy.spatial import cKDTree

In [128]:
def crossmatch(X1, X2, max_distance=np.inf):

    X1 = np.asarray(X1, dtype=float)
    X2 = np.asarray(X2, dtype=float)

    N1, D = X1.shape
    N2, D2 = X2.shape

    if D != D2:
        raise ValueError('Arrays must have the same second dimension')

    kdt = cKDTree(X2)
    
    dist, ind = kdt.query(X1,k=10, distance_upper_bound=max_distance)

    return dist, ind

In [129]:
def crossmatch_angular(X1, X2, max_distance):
    
    X1 = X1 * (np.pi / 180.)
    X2 = X2 * (np.pi / 180.)
    max_distance = max_distance * (np.pi / 180.)

    # Convert 2D RA/DEC to 3D cartesian coordinates
    Y1 = np.transpose(np.vstack([np.cos(X1[0]) * np.cos(X1[1]),
                                 np.sin(X1[0]) * np.cos(X1[1]),
                                 np.sin(X1[1])]))
    Y2 = np.transpose(np.vstack([np.cos(X2[:, 0]) * np.cos(X2[:, 1]),
                                 np.sin(X2[:, 0]) * np.cos(X2[:, 1]),
                                 np.sin(X2[:, 1])]))

    # law of cosines to compute 3D distance
    max_y = np.sqrt(2 - 2 * np.cos(max_distance))
    dist, ind = crossmatch(Y1, Y2, max_y)

    # convert distances back to angles using the law of tangents
    not_inf = ~np.isinf(dist)
    x = 0.5 * dist[not_inf]
    dist[not_inf] = (180. / np.pi * 2 * np.arctan2(x,
                                  np.sqrt(np.maximum(0, 1 - x ** 2))))

    return dist, ind

In [ ]:
def r2arcsec(r):
    return r * 180 / np.pi


dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/pair-m-z/gal_info_dr7_v5_2.fit'
dir_indo= '/Users/astro/Documents/notebooks/zs/pair-m-z/cross-match_query_k_10.txt'
with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7,open(dir_indo,'a+') as f:
    data = f_drp[1].data

    manga = np.empty((len(data), 2), dtype=np.float64)
    manga[:, 0] = data['objra']
    manga[:, 1] = data['objdec']
    manga_z = data['z']
    plateifu=data['plateifu']

    data1 = f_dr7[1].data
    dr7 = np.empty((len(data1), 2), dtype=np.float64)
    dr7[:, 0] = data1['RA']
    dr7[:, 1] = data1['DEC']
    dr7_z = data1['Z']

    for i in range(0, len(manga)):
        z = manga_z[i]
        dis = cosmo.comoving_distance(z)  # Mpc
        r = 0.15 / dis.value
        max_r = r2arcsec(r)  # degree

        dist, ind = crossmatch_angular(manga[i], dr7, max_r)
        match = ~np.isinf(dist)
        dist_match = dist[match]
        
        print(plateifu[i],manga[i],z,dr7[ind-1],dr7_z[ind-1],file=f)

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]


In [ ]:
# from PyAstronomy import pyasl

In [ ]:
# pyasl.getAngDist(133.37095642,57.59861755,133.37109061,57.59842514)

In [ ]:
# a

In [152]:
from astropy import units as u
from astropy.coordinates import Distance

d=Distance(z=0.15)

In [153]:
d

<Distance 736.22628844 Mpc>

In [154]:
arc=3*np.pi/(3600*180)

In [155]:
arc

1.4544410433286079e-05

In [156]:
d*arc*2

<Distance 0.02141595 Mpc>

In [148]:
206265/300

687.55